In [1]:
import pandas as pd
import numpy as np
import pymrio
import gzip

In [2]:
# select the current version of ecoinvent as well the one the new one for which concordances will be updated
current_version = '3.6'
new_version = '3.7.1'

In [3]:
# load ecoinvent
with gzip.open('C://Users/Maxime/Desktop/Thesis/Bdd/Ecoinvent/python/ecoinvent'+new_version+'.cutoff/ecoinvent'+new_version+'.cut-offPandas_symmNorm.gz.pickle','rb') as f:
    ecoinvent = pd.read_pickle(f)
# load the report of changes from ecoinvent
change_report = pd.read_excel('C://Users/Maxime/Desktop/change_report_annex_'+current_version+'_'+new_version+'_public.xlsx','qualitative changes')
# load previous version of concordance file
prod_concordance = pd.read_excel('C://Users/Maxime/Desktop/Thesis/Modules_Python/pylcaio/src/Data/eco'+current_version+'_exio3/Product_Concordances.xlsx','Concordance per product')
act_concordance = pd.read_excel('C://Users/Maxime/Desktop/Thesis/Modules_Python/pylcaio/src/Data/eco'+current_version+'_exio3/Product_Concordances.xlsx','Concordance per activity')

#### For products

In [4]:
# selecting processes for which the reference product changed
try:
    if "FALSE" in change_report.loc[:,'reference product different'].tolist():
        changed_names = change_report.loc[[i for i in change_report.index if change_report.loc[i,'reference product different']=='TRUE']]
    elif False in change_report.loc[:,'reference product different'].tolist():
        changed_names = change_report.loc[[i for i in change_report.index if change_report.loc[i,'reference product different']==True]]
except KeyError:
    print('Column used got renamed.')
    
# extracting the changed data as a dictionary
try:
    names_dict = dict(zip(changed_names.loc[:,'reference product - '+current_version],changed_names.loc[:,'reference product - '+new_version]))
except KeyError:
    print('Column used got renamed.')

# make the changes inside the file
prod_concordance.loc[:,'productName'] = [names_dict[i] if i in names_dict else i for i in prod_concordance.productName]

# get the product ids from ecoinvent
updated_prod_concordance = prod_concordance.loc[:,['productName','Concordance']].merge(ecoinvent['PRO'].loc[:,['productId','productName']], how='left').drop_duplicates()

#reordering
updated_prod_concordance = updated_prod_concordance.reindex(['productId','productName','Concordance'],axis=1)

# Now copy the Product_concordance.xlsx file and replace the tab "Concordance per product" with this dataframe

#### For activities

In [5]:
# selecting processes for which the reference product changed
try:
    if "FALSE" in change_report.loc[:,'activityName different'].tolist():
        changed_names = change_report.loc[[i for i in change_report.index if change_report.loc[i,'activityName different']=='TRUE']]
    elif False in change_report.loc[:,'activityName different'].tolist():
        changed_names = change_report.loc[[i for i in change_report.index if change_report.loc[i,'activityName different']==True]]
except KeyError:
    print('Column used got renamed.')

# extracting the changed data as a dictionary
try:
    names_dict = dict(zip(changed_names.loc[:,'activityName - '+current_version],changed_names.loc[:,'activityName - '+new_version]))
except KeyError:
    print('Column used got renamed.')

# make the changes inside the file
act_concordance.loc[:,'activityName'] = [names_dict[i] if i in names_dict else i for i in act_concordance.activityName]

# get the product ids from ecoinvent
updated_act_concordance = act_concordance.loc[:,['activityName','Concordance']].merge(ecoinvent['PRO'].loc[:,['activityNameId','activityName']], how='left').drop_duplicates()

#reordering
updated_act_concordance = updated_act_concordance.reindex(['activityNameId','activityName','Concordance'],axis=1)

# Now copy the Product_concordance.xlsx file and replace the tab "Concordance per activity" with this dataframe

#### New processes

In [10]:
# load filter for hybridization
filterhyb = pd.read_excel('C://Users/Maxime/Desktop/Thesis/Modules_Python/pylcaio/src/Data/eco'+new_version+'_exio3/Filter.xlsx')

In [11]:
# identify new processes that will be hybridized. The ones that won't be hybridized don't matter
new_processes = filterhyb.loc[[i for i in filterhyb.index if (filterhyb.loc[i,'productName'] not in updated_prod_concordance.productName.tolist() and
                               filterhyb.loc[i,'activityName'] not in updated_act_concordance.activityName.tolist())]]

In [12]:
# now complete this dataframe (by hand), i.e., decide which exiobase sector corresponds to those new processes
# and append these processes to the existing "Concordance per product" tab
concordances_to_do = ecoinvent['PRO'].loc[new_processes.loc[:,'index'], ['productId','productName']].reset_index().drop(['index'],axis=1)
concordances_to_do = concordances_to_do.drop_duplicates()